# Investigating the relationship between the 'basic_reproduction_number' parameter and Rt.

The effective reproduction number characterizes the temporal dynamics of an infectious disease.

Here I apply the bayesian inference method from BranchPro to inferr the effective R0 from simulation outputs.

The main parameter required by these models is the serial interval. Since we aim for an overall effective R0 of approximately 3, I searched the literature for information on the serial interval distribution for the original SARS-CoV-2 variant. Estimates of the mean serial interval for this period range from 4.2 to 7.5. In addition, the study design varies quite substantially. On the basis of its use in Menendez (2020), I proceed with an estimated gaussian serial distribution with mean = 4.7 and standard deviation = 2.9 when using fixed serial intervals.

In [4]:
SI_mean = 4.7
SI_sd = 2.9

We use the simulation outputs from 9 populations:
- 4x4 grid with population of 10000
- 8x8 grid with population of 10000
- 12x12 grid with population of 10000
- Gibralter simulation with spatial infections in neighbour cell
- Gibralter simulation with spatial infections across the whole country

with the first 4 being run with 'basic_reproduction_number' being equal to 3 or 1.5.


In [27]:
import pandas as pd
import numpy as np
import branchpro
import scipy.stats
import statistics

In [24]:

# Read in SIR curves
# Toy populations of size = 10,000 with 'basic_reproduction_number' = 3:
df = pd.read_csv("combined_summary.csv")

# Gibralter with 'basic_reproduction_number' = 3 and spatial radius = 0.0075:
df2 = pd.read_csv("NewEpiabm/epiabm/python_examples/gibraltar_example/simulation_outputs/small_rad_summary.csv")

# Gibralter with 'basic_reproduction_number' = 3 and spatial radius = :
df3 = pd.read_csv("NewEpiabm/epiabm/python_examples/gibraltar_example/simulation_outputs/large_rad_summary.csv")

# Toy populations of size = 10,000 with 'basic_reproduction_number' = 1.5:
df4 = pd.read_csv("NewEpiabm/epiabm/python_examples/toy_spatial/simulation_outputs/R0/combined_summary_r1.5.csv")

# Gibralter with 'basic_reproduction_number' = 1.5 and spatial radius = 0.0075:
df5 = pd.read_csv("NewEpiabm/epiabm/python_examples/gibraltar_example/simulation_outputs/r1_5_summary.csv")

data_all = pd.DataFrame(data={'high_dense_3': df['av_susceptible_4'],
                              'mid_dense_3': df['av_susceptible_8'],
                              'low_dense_3': df['av_susceptible_12'],
                              'high_dense_1_5': df4['av_susceptible_4'],
                              'mid_dense_1_5': df4['av_susceptible_8'],
                              'low_dense_1_5': df4['av_susceptible_12'],
                              'gibralter_3': df2['Susceptible'],
                              'gibralter_1_5': df5['Susceptible'],
                              'gibralter_3_large_spatial': df3['Susceptible']})

# Read serial interval
serial_interval = pd.read_csv("si-covid.csv", header=None)
serial_interval = serial_interval.fillna(0)
serial_intervals = serial_interval.values.T

First we run the inference for the toy populations with R0 = 3 and then R0 = 1.5:

In [28]:
labels = ['4x4, r=3', '8x8, r=3', '12x12, r=3', '4x4, r=1.5', '8x8, r=1.5', '12x12, r=1.5']


for i in range(6):
    print(labels[i])
    x = data_all.iloc[:,i]
    locally_infected_cases = -np.diff(x.values)[:20]
    locally_infected_cases = [10] + list(locally_infected_cases)


    # Build the serial interval w_s
    num_timepoints = len(locally_infected_cases)
    ws_mean = SI_mean
    ws_var = SI_sd**2
    theta = ws_var / ws_mean
    k = ws_mean / theta
    w_dist = scipy.stats.gamma(k, scale=theta)
    disc_w = w_dist.pdf(np.arange(num_timepoints))


    # Same inference, but using the LocImpBranchProPosterior
    tau = 3
    R_t_start = tau+1
    a = 1
    b = 1/5


    # Transform our incidence data into pandas dataframes
    inc_data = pd.DataFrame(
        {
            'Time': np.arange(num_timepoints),
            'Incidence Number': locally_infected_cases
        }
    )

    inference = branchpro.BranchProPosteriorMultSI(
        inc_data=inc_data, 
        daily_serial_intervals=serial_intervals,
        alpha=a,
        beta=b)
    
    # inference = branchpro.BranchProPosterior(
    #     inc_data=inc_data,
    #     daily_serial_interval=disc_w,
    #     alpha=a,
    #     beta=b)


    inference.run_inference(tau=tau)
    intervals = inference.get_intervals(central_prob=.95)


    # Inference plot using class method results
    fig = branchpro.ReproductionNumberPlot()
    fig.add_interval_rt(intervals)
    fig.update_labels(time_label='Time (Day)', r_label='R_t')
    fig.show_figure()

    first_week = intervals['Mean'][:7]

    print('Average Rt over the first week = ', statistics.mean(first_week))

4x4, r=3


Average Rt over the first week =  4.749209886920302
8x8, r=3


Average Rt over the first week =  3.701679695114752
12x12, r=3


Average Rt over the first week =  3.1322153891670586
4x4, r=1.5


Average Rt over the first week =  4.802394077101385
8x8, r=1.5


Average Rt over the first week =  3.839021951996583
12x12, r=1.5


Average Rt over the first week =  3.5815623290061254


In [32]:
labels = ['Gibralter, r=3', 'Gibralter, r=1.5', 'Gibralter, r=3 with large spatial radius']


for i in [6, 7, 8]:
    print(labels[i-6])
    x = data_all.iloc[:,i]
    locally_infected_cases = -np.diff(x.values)[:20]
    locally_infected_cases = [100] + list(locally_infected_cases)


    # Build the serial interval w_s
    num_timepoints = len(locally_infected_cases)
    ws_mean = SI_mean
    ws_var = SI_sd**2
    theta = ws_var / ws_mean
    k = ws_mean / theta
    w_dist = scipy.stats.gamma(k, scale=theta)
    disc_w = w_dist.pdf(np.arange(num_timepoints))


    # Same inference, but using the LocImpBranchProPosterior
    tau = 3
    R_t_start = tau+1
    a = 1
    b = 1/5


    # Transform our incidence data into pandas dataframes
    inc_data = pd.DataFrame(
        {
            'Time': np.arange(num_timepoints),
            'Incidence Number': locally_infected_cases
        }
    )

    inference = branchpro.BranchProPosteriorMultSI(
        inc_data=inc_data, 
        daily_serial_intervals=serial_intervals,
        alpha=a,
        beta=b)
    
    # inference = branchpro.BranchProPosterior(
    #     inc_data=inc_data,
    #     daily_serial_interval=disc_w,
    #     alpha=a,
    #     beta=b)


    inference.run_inference(tau=tau)
    intervals = inference.get_intervals(central_prob=.95)


    # Inference plot using class method results
    fig = branchpro.ReproductionNumberPlot()
    fig.add_interval_rt(intervals)
    fig.update_labels(time_label='Time (Day)', r_label='R_t')
    fig.show_figure()

    first_week = intervals['Mean'][:7]

    print('Average Rt over the first week = ', statistics.mean(first_week))

Gibralter, r=3


Average Rt over the first week =  2.273195875239228
Gibralter, r=1.5


Average Rt over the first week =  2.273195875239228
Gibralter, r=3 with large spatial radius


Average Rt over the first week =  2.273195875239228


Here, we estimate the Rt profile for the population used in the estimates of the true R0. Expected R0 here = 2.8.

In [44]:

df6 = pd.read_csv('NewEpiabm/epiabm/python_examples/age_stratified_example/simulation_outputs/output_with_age.csv')

df6 = df6.drop(["InfectionStatus.Exposed",
                              "InfectionStatus.InfectASympt",
                              "InfectionStatus.InfectGP",
                              "InfectionStatus.InfectHosp",
                              "InfectionStatus.InfectICU",
                              "InfectionStatus.InfectICURecov",
                              "InfectionStatus.Dead"],
                             axis=1)
df6 = df6.groupby(["time"]).agg(
                                {"InfectionStatus.Susceptible": 'sum',
                                 "InfectionStatus.InfectMild": 'sum',
                                 "InfectionStatus.Recovered": 'sum'})

x = df6['InfectionStatus.Susceptible']
locally_infected_cases = -np.diff(x.values)[:20]
locally_infected_cases = [1] + list(locally_infected_cases)


# Build the serial interval w_s
num_timepoints = len(locally_infected_cases)
ws_mean = SI_mean
ws_var = SI_sd**2
theta = ws_var / ws_mean
k = ws_mean / theta
w_dist = scipy.stats.gamma(k, scale=theta)
disc_w = w_dist.pdf(np.arange(num_timepoints))


# Same inference, but using the LocImpBranchProPosterior
tau = 3
R_t_start = tau+1
a = 1
b = 1/5


# Transform our incidence data into pandas dataframes
inc_data = pd.DataFrame(
    {
        'Time': np.arange(num_timepoints),
        'Incidence Number': locally_infected_cases
    }
)

inference = branchpro.BranchProPosteriorMultSI(
    inc_data=inc_data, 
    daily_serial_intervals=serial_intervals,
    alpha=a,
    beta=b)

# inference = branchpro.BranchProPosterior(
#     inc_data=inc_data,
#     daily_serial_interval=disc_w,
#     alpha=a,
#     beta=b)


inference.run_inference(tau=tau)
intervals = inference.get_intervals(central_prob=.95)


# Inference plot using class method results
fig = branchpro.ReproductionNumberPlot()
fig.add_interval_rt(intervals)
fig.update_labels(time_label='Time (Day)', r_label='R_t')
fig.show_figure()

first_week = intervals['Mean'][:7]

print('Average Rt over the first week = ', statistics.mean(first_week))

Average Rt over the first week =  1.6985483583157428
